In [ ]:
import gradio as gr
import re
import io
import base64
from matplotlib import pyplot as plt

def process_text_with_code_blocks(text: str) -> str:
    code_blocks = re.findall(r"```python(.*?)```", text, re.DOTALL)

    for code_block in code_blocks:
        code = code_block.strip()
        img_buffer = io.BytesIO()
        local_env = {"plt": plt, "io": io}

        try:
            exec(code, local_env)
            plt.savefig(img_buffer, format='png')
            plt.close()
            img_buffer.seek(0)

            img_base64 = base64.b64encode(img_buffer.read()).decode("utf-8")
            image_markdown = f"![Biểu đồ](data:image/png;base64,{img_base64})"
            text = text.replace(f"```python\n{code}\n```", image_markdown)
        except Exception as e:
            text = f"Lỗi khi chạy code: {e}"
    return text

# Tạo giao diện Gradio
demo = gr.Interface(
    fn=process_text_with_code_blocks,
    inputs=gr.Textbox(lines=20, label="Nhập văn bản có chứa code Python", placeholder="Nhập văn bản có code ở dạng ```python ... ```"),
    outputs=gr.Markdown(label="Văn bản sau khi gắn biểu đồ"),
    title="Biến Code Thành Biểu Đồ Tự Động",
    description="Nhập văn bản chứa đoạn code Python trong khối ```python ... ``` để tự động chạy code và gắn biểu đồ vào vị trí tương ứng."
)

demo.launch()


In [ ]:
import gradio as gr
import re
import matplotlib.pyplot as plt
import uuid
import os
from PIL import Image

def process_message(history, message):
    history.append((message, None))

    code_blocks = re.findall(r"```python(.*?)```", message, re.DOTALL)

    if not code_blocks:
        history[-1] = (message, "🤖 Không tìm thấy đoạn mã Python nào.")
        return history, None

    for code in code_blocks:
        code = code.strip()
        img_path = f"temp_{uuid.uuid4().hex}.png"
        local_env = {"plt": plt}

        try:
            exec(code, local_env)
            plt.savefig(img_path)
            plt.close()

            # Đọc ảnh và trả về đường dẫn
            history.append((None, gr.Image(img_path, label="Biểu đồ Python")))
        except Exception as e:
            history.append((None, f"🤖 Lỗi khi chạy code:\n```\n{e}\n```"))

    return history, ""

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Trợ lý Biểu đồ Python", bubble_full_width=False)
    msg = gr.Textbox(label="Nhập văn bản có chứa code Python", placeholder="Nhập text có chứa ```python ...```")
    state = gr.State([])

    msg.submit(process_message, [state, msg], [chatbot, msg])
    msg.submit(lambda: "", None, msg)

demo.launch()


C:\Users\PC\AppData\Local\Temp\ipykernel_2696\2841669594.py:35: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Trợ lý Biểu đồ Python", bubble_full_width=False)
C:\Users\PC\AppData\Local\Temp\ipykernel_2696\2841669594.py:35: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(label="Trợ lý Biểu đồ Python", bubble_full_width=False)


* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


<string>:16: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
<string>:15: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown


*** 
```python
[code]
```

In [12]:
import gradio as gr
import re
import matplotlib.pyplot as plt
import uuid
from PIL import Image

def process_message(history, message, images):
    history.append((message, None))

    code_blocks = re.findall(r"```python(.*?)```", message, re.DOTALL)

    if not code_blocks:
        history[-1] = (message, "🤖 Không tìm thấy đoạn mã Python nào.")
        return history, images

    for code in code_blocks:
        code = code.strip()
        img_path = f"temp_{uuid.uuid4().hex}.png"
        local_env = {"plt": plt}

        try:
            exec(code, local_env)
            plt.savefig(img_path)
            plt.close()

            # Đọc ảnh vào bộ nhớ và thêm vào danh sách ảnh
            img = Image.open(img_path)
            images.append(img)

        except Exception as e:
            history.append((None, f"🤖 Lỗi khi chạy code:\n```\n{e}\n```"))

    # Trả về lịch sử và tất cả ảnh đã tạo
    return history, images

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Trợ lý Biểu đồ Python", bubble_full_width=False)
    msg = gr.Textbox(label="Nhập văn bản có chứa code Python", placeholder="Nhập text có chứa ```python ...```")
    state = gr.State([])  # Lưu trữ lịch sử chat
    img_state = gr.State([])  # Lưu trữ danh sách ảnh đã được tạo ra

    # Sử dụng gr.Gallery để hiển thị tất cả các ảnh
    gallery = gr.Gallery(label="Các biểu đồ Python")

    # Xử lý tin nhắn và thêm ảnh vào trạng thái
    msg.submit(process_message, [state, msg, img_state], [chatbot, gallery])

    msg.submit(lambda: "", None, msg)

demo.launch()


C:\Users\PC\AppData\Local\Temp\ipykernel_2696\3571152332.py:37: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Trợ lý Biểu đồ Python", bubble_full_width=False)
C:\Users\PC\AppData\Local\Temp\ipykernel_2696\3571152332.py:37: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(label="Trợ lý Biểu đồ Python", bubble_full_width=False)


* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.
